In [1]:
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/Colab Notebooks/Project 0/Project 0/Notebooks"
import sys
sys.path.append(path)
from Analysis.Regression_Analysis.Regression import *
from Analysis.Tools.Loading import *
from Analysis.Tools.Training import *
import json


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Training Baseline Models

In [2]:
positive_class = ['AP','DP']
max_fixations = 20
phases=['Learning','Target','Distractor']
results = {}
data_path = "/content/drive/MyDrive/Colab Notebooks/Project 0/Project 0/Notebooks/Data"


In [ ]:

results = {}
for pc in positive_class:      
          
       # import data from csv files 
      labels = {'Positive_Class':f'{pc} - New ROI Data','Negative_Class':'Control - New ROI Data'}

      data = EyeTrackingData(data_path = data_path,labels=labels,phases=phases).Load()
      positive_subjects = sorted(list(data[data.LABEL==1]['RECORDING_SESSION_LABEL'].unique()))
      negative_subjects = sorted(list(data[data.LABEL==0]['RECORDING_SESSION_LABEL'].unique()))
    
      train_subjects, test_subjects = RandomTrainTestSplit(positive_subjects=positive_subjects,negative_subjects=negative_subjects,percent_val = None,percent_test = 0.3)
      
      # train models for 1<fixations<=20
      results[pc] = {'num_fixations':[],'score':[]}
      
      for num_fixations in range(2,max_fixations+1):
          print(f'using {num_fixations} fixations (max={max_fixations})')
          # edit dataframe to include up to nth fixation
          trials = LoadTrials(data,num_fixations)
          # process the features to make the data ready for training
          df = Preprocess(trials,num_fixations)
          # record the fixation number and auc for the plot
          
          results[pc]['num_fixations'].append(num_fixations)
          results[pc]['score'].append(TrainClassifier(df,train_subjects, test_subjects,upsample=True)) #upsample = True will copy the data for the positive class. Should not be used if dataset is balanced
          #results[pc]['phase'].append(phase)

In [ ]:
with open(os.path.join(path, 'Analysis/Regression_Analysis/Results.json'), "w") as write_file:
    json.dump(results, write_file)